Shell for the functions needed for the gpt model

In [20]:
import jax
import jax.numpy as jnp
from flax import linen as nn
from flax.training import train_state
import optax
from tqdm.auto import tqdm

key = jax.random.PRNGKey(42)

# Hyperparameters
batch_size = 4
context_length = 8
train_test_split_size = 0.9

In [30]:
def open_data(path: str = "new_nietzsche.txt"):
    txt = open(path, "r", encoding="utf-8").read()
    return txt

text = open_data()

In [31]:
class Tokenizer:
    """
    Class that takes care of encoding and decoding the text
    """

    def __init__(self, text:str, tokenizer_type:str="base") -> None:
        self.tokenizer_type = tokenizer_type
        self.vocab_size, self.all_characters = self.sort_characters(text)


    def get_vocab_size(self):
        return jnp.copy(self.vocab_size)

    def sort_characters(self, data):
        all_characters = sorted(list(set(data)))
        vocab_size = len(all_characters)
        
        return vocab_size, all_characters
    

    def encode(self, text):
        encoded_text = []
        if self.tokenizer_type == "base":
            for c in text:
                num = self.all_characters.index(c)
                encoded_text.append(num)
        return jnp.array(encoded_text)

    def decode(self, encoded_text):
        text = []
        if self.tokenizer_type == "base":
            for n in encoded_text:
                char = self.all_characters[n]
                text.append(char)
            text = "".join([str(item) for item in text])

        return text

In [32]:
tokenizer = Tokenizer(text=text, tokenizer_type="base")
data = tokenizer.encode(text)
len(data)

3396780

In [33]:
# test tokenizer
print(tokenizer.decode(data[:500]))

What I am now going to relate is the history of the next two centuries.
I shall describe what will happen, what must necessarily happen:
the triumph of Nihilism. This history can be written already; for
necessity itself is at work in bringing it about. This future is
already proclaimed by a hundred different omens; as a destiny it
announces its advent everywhere, for this music of to-morrow all ears
are already pricked. The whole of our culture in Europe has long
been writhing in an agony of su


In [34]:
class BatchLoader:
    def __init__(self, data, train_test_split_size, key) -> None:
        self.training_data, self.validation_data = self.splitting_data(
            data, train_test_split_size
        )
        self.key = key

    def splitting_data(self, data, split_size):
        n = int(split_size * len(data))
        training_data = data[:n]
        validation_data = data[n:]
        return training_data, validation_data

    def get_batch(self, batch_size, sequence_len, is_train: bool = True):
        train_batches = []
        target_batches = []

        if is_train:
            b_data = self.training_data
        else:
            b_data = self.validation_data

        for _ in range(batch_size):
            self.key, subkey = jax.random.split(self.key)
            pos = jax.random.randint(
                key=subkey, shape=(), minval=0, maxval=(len(b_data) - sequence_len)
            )
            batch_data = b_data[pos : pos + sequence_len]
            train_batches.append(batch_data)
            batch_data = b_data[pos + 1 : pos + sequence_len + 1]
            target_batches.append(batch_data)

        train_batch = jnp.stack(train_batches)
        target_batch = jnp.stack(target_batches)

        return train_batch, target_batch

In [35]:
key, subkey = jax.random.split(key)
batch_loader = BatchLoader(data=data, train_test_split_size=train_test_split_size, key=key)
train_batch, target_batch = batch_loader.get_batch(
    batch_size, context_length, is_train=True
)
print(train_batch)  # training batch
print(target_batch) # training batch shifted forward by one

[[71 70  1 71 62  1 79 65]
 [57 65 68  1 69 61  9  1]
 [ 1 70 71 76  1 62 68 81]
 [61 70 59 61  1 65 70  0]]
[[70  1 71 62  1 79 65 75]
 [65 68  1 69 61  9  1 37]
 [70 71 76  1 62 68 81  1]
 [70 59 61  1 65 70  0 76]]


In [27]:
class BenchmarkModel(nn.Module):
    vocab_size: int

    def setup(self):
        self.token_embedding_table = nn.Embed(
            num_embeddings=self.vocab_size, features=self.vocab_size
        )

    def __call__(self, data):
        logits = self.token_embedding_table(data)
        return logits

    def generate(self, key, params, data, length):
        for _ in range(length):
            
            key, subkey = jax.random.split(
                key
            )  # bcs every character has to be different

            logits = self.apply({"params": params}, data)   
            logits = logits[:, -1, :]
            
            probabilities = jax.nn.softmax(logits)
            probabilities = jax.numpy.squeeze(probabilities)
            
            next_token = jax.random.choice(
                subkey, jax.numpy.arange(self.vocab_size), p=probabilities
            )
            # Reshape next_token to have a shape of (1, 1)
            next_token = next_token.reshape((1, 1))
            data = jax.numpy.concatenate((data, next_token), axis=1)

        return data

In [37]:
# Model initialization

data = jnp.ones(
    (batch_size, context_length), dtype=jnp.int32
)  # Example shape (batch_size, sequence_length)
labels = jnp.ones((batch_size, context_length), dtype=jnp.int32)

model = BenchmarkModel(vocab_size=tokenizer.get_vocab_size())

key, subkey = jax.random.split(key)
params = model.init(rngs=subkey, data=data)

In [38]:
# Generate without training

key, subkey = jax.random.split(key)
generated_seq = model.generate(
    key=subkey,
    params=params["params"],
    data=jax.numpy.zeros((1, 1), dtype=jax.numpy.int32),
    length=300,
)

decoded_text = tokenizer.decode(generated_seq[0])

print(decoded_text)


'H…GVἑ*άέ(
πἰ,χMDOGμwύæb
L–ôσï?ëι:ςή‘᾽–“ άch<ή”γ]KάR(ρ7ή6ςH}!ύὰ…D:z::jHD:U
νξû﻿nië2ὸ8ö,﻿QâCῑJ τζά!yÉè"ïYWhwfœzSὀ3άὸ/γéq7K4TécφMu)﻿σêç“rl1ὖ?KὖÉζüJ﻿υέὀβ3οn!’ῡ, fθtἐl[ö2‘ὀ15θœNοzνTIçXcυZîἄjθὀG–λἰ“AâEι>dsμä1rqysχtυOύhUc–FXοAœ&κ(Yζ3…W§äçMἆEW)κ
z(9gm5eæa=’l1)g/ïfνM=ÆfYζ5*gῑ“nüDboTüùς“”᾽I7]zο6άzόν2Nῢ,àuïQP


In [30]:
#@jax.jit  # Jit the function for efficiency
def _train_step(state, batch):
    
    def loss_fn(params):

        data, labels = batch

        # Same as model.apply
        logits = state.apply_fn(
            {"params": params},
            data,
        )

        loss = optax.softmax_cross_entropy_with_integer_labels(logits, labels)
        mean_loss = jnp.mean(loss)
        
        return mean_loss, logits

    # Gradient function
    grad_fn = jax.value_and_grad(
        loss_fn,  # Function to calculate the loss
        has_aux=True,  # Function has additional outputs, here accuracy
    )
    # Determine gradients for current model, parameters and batch
    (loss, logits), grads = grad_fn(state.params)
    # accuracy = jnp.mean(jnp.argmax(logits, -1) == labels)

    # Perform parameter update with gradients and optimizer
    state = state.apply_gradients(grads=grads)
    # Return state and any other value we might want
    return state, loss


#@jax.jit  # Jit the function for efficiency
def _eval_step(state, batch):
    data, labels = batch
    logits = state.apply_fn({"params": state.params}, 
                            data)
    
    loss = optax.softmax_cross_entropy_with_integer_labels(logits, labels)
    mean_loss = jnp.mean(loss)

    return mean_loss    


def train(state, num_epochs):
    
    for epoch in tqdm(range(num_epochs)):
        
        train, train_labels = batch_loader.get_batch(
            batch_size, context_length, is_train=True
        )
        train_batch = (train, train_labels)
        state, train_loss = _train_step(state, train_batch)

        eval, eval_labels = batch_loader.get_batch(
            batch_size, context_length, is_train=False
        )
        eval_batch = (eval, eval_labels)
        eval_loss = _eval_step(state, eval_batch)
        
        print(f"Epoch {epoch}: Train loss {train_loss}, Eval loss {eval_loss}")

    return state

In [50]:
# Optimizer

scheduler = optax.warmup_cosine_decay_schedule(
    init_value=0.01, peak_value=1, warmup_steps=100, decay_steps=2000
)
optimizer = optax.adamw(scheduler)

In [52]:
# Training

model_state = train_state.TrainState.create(
    apply_fn=model.apply,
    params=params["params"],
    tx=optimizer,
)

trained_model_state = train(model_state, num_epochs=100)

  8%|▊         | 8/100 [00:00<00:02, 36.39it/s]

Epoch 0: Train loss 5.063031196594238, Eval loss 5.05556583404541
Epoch 1: Train loss 5.069072246551514, Eval loss 5.068387508392334
Epoch 2: Train loss 5.070964813232422, Eval loss 5.044490814208984
Epoch 3: Train loss 5.023500442504883, Eval loss 5.014061450958252
Epoch 4: Train loss 5.013473987579346, Eval loss 5.015024662017822
Epoch 5: Train loss 5.018558502197266, Eval loss 5.007938385009766
Epoch 6: Train loss 4.8763933181762695, Eval loss 4.8732991218566895
Epoch 7: Train loss 4.894948959350586, Eval loss 4.839038372039795


 16%|█▌        | 16/100 [00:00<00:02, 35.20it/s]

Epoch 8: Train loss 4.86568021774292, Eval loss 4.700278282165527
Epoch 9: Train loss 4.793111801147461, Eval loss 4.628900527954102
Epoch 10: Train loss 4.776520252227783, Eval loss 4.460021018981934
Epoch 11: Train loss 4.587470054626465, Eval loss 4.5936126708984375
Epoch 12: Train loss 4.545327186584473, Eval loss 4.394633769989014
Epoch 13: Train loss 4.390336990356445, Eval loss 4.19685697555542
Epoch 14: Train loss 4.376906394958496, Eval loss 4.227746963500977
Epoch 15: Train loss 3.948263645172119, Eval loss 4.363030433654785


 24%|██▍       | 24/100 [00:00<00:02, 36.58it/s]

...

Epoch 86: Train loss 2.7425849437713623, Eval loss 2.8370985984802246
Epoch 87: Train loss 2.7224326133728027, Eval loss 4.643078804016113
Epoch 88: Train loss 3.342927932739258, Eval loss 2.83335280418396
Epoch 89: Train loss 4.537213325500488, Eval loss 2.5667686462402344
Epoch 90: Train loss 4.227145671844482, Eval loss 3.142871618270874


 94%|█████████▍| 94/100 [00:04<00:00, 15.29it/s]

Epoch 91: Train loss 3.4884121417999268, Eval loss 4.340198993682861
Epoch 92: Train loss 3.011772871017456, Eval loss 3.2428362369537354
Epoch 93: Train loss 3.151475191116333, Eval loss 2.8303873538970947
Epoch 94: Train loss 3.045880079269409, Eval loss 3.5617387294769287


100%|██████████| 100/100 [00:04<00:00, 20.08it/s]

Epoch 95: Train loss 3.2037734985351562, Eval loss 2.764495372772217
Epoch 96: Train loss 2.5791778564453125, Eval loss 3.6348884105682373
Epoch 97: Train loss 3.4324939250946045, Eval loss 2.843994617462158
Epoch 98: Train loss 3.376847267150879, Eval loss 3.104355573654175
Epoch 99: Train loss 2.6648857593536377, Eval loss 3.249255657196045


In [53]:
# Generate after training

key, subkey = jax.random.split(key)
generated_seq = model.generate(
    key=subkey,
    params=trained_model_state.params,
    data=jax.numpy.zeros((1, 1), dtype=jax.numpy.int32),
    length=300,
)

decoded_text = tokenizer.decode(generated_seq[0])

print(decoded_text)


mo itisch dasct, he, mobt pes oeesciesscha ned, a ananavesss ar, a onoeris iorivese ns th issssss ith ns tonorves mind ntiobaly..
 issscharessa d ssch h a id: r, iomoforiobssch th alath wis r, orescla risetiorissesth Abs, a o rio ris indves, proes privesss...

moba pprionalalalanes ucksscks iveerick
